In [13]:
import os
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher
from gensim.models import KeyedVectors
import tempfile
import pandas as pd
import numpy as np
import h5py, torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator, CosComparator
import json

In [14]:
# Parameters

current_path = os.getcwd()

tutorial_deployment_path = current_path[0:len(current_path)-9] + '/datasets'

# Folder on local machine where to create the output and temporary folders
input_path = tutorial_deployment_path + '/arnold-profiled'
# input_path = "/Users/pedroszekely/Documents/GitHub/kgtk-tutorial-files/datasets/arnold-profiled"

output_path = "/tmp/projects"
# output_path = "/Users/pedroszekely/Downloads/kypher/projects"

project_name = "tutorial-graph-embeddings"

In [15]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "item",
    "qualifiers",
    "p31",
    "p279star"
]
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /Users/grantxie
Current dir: /Users/grantxie/Downloads/kgtk-notebooks/tutorial
KGTK dir: /Users/grantxie/Downloads/kgtk-notebooks
Use-cases dir: /Users/grantxie/Downloads/kgtk-notebooks/use-cases


In [16]:
ck.print_env_variables()

TEMP: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings
GRAPH: /Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled
kgtk: kgtk
STORE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
EXAMPLES_DIR: /Users/grantxie/Downloads/kgtk-notebooks/examples
OUT: /tmp/projects/tutorial-graph-embeddings
KGTK_LABEL_FILE: /Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/labels.en.tsv.gz
USE_CASES_DIR: /Users/grantxie/Downloads/kgtk-notebooks/use-cases
KGTK_OPTION_DEBUG: false
KGTK_GRAPH_CACHE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
kypher: kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
all: /Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/all.tsv.gz
label: /Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/labels.en.tsv.gz
alias: /Users/grantxie/Downloads/kgt

In [17]:
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/all.tsv.gz" --as all  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/labels.en.tsv.gz" --as label  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/aliases.en.tsv.gz" --as alias  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/descriptions.en.tsv.gz" --as description  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/claims.wikibase-item.tsv.gz" --as item  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/qualifiers.tsv.gz" --as qualifiers  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/derived.P31.tsv.gz" --as p31  -i "/Users/grantxie/Downloads/kgtk-notebooks/datasets/arnold-profiled/derived.P279star.tsv.gz" --as p279star  --limit 3
node1	label	node2	id	node2;wikidatatype
P10

In [18]:
vector_dimension = 30
vector_output_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.tsv"
vector_output_w2v_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.w2v.tsv"
os.environ['VECTOR_DIMENSION'] = str(vector_dimension)

## Compute ComplEx Graph Embeddings

In this notebook we will compute graph embeddings using `kgtk graph-embeddings` command for the `arnold` subgraph and demonstrate a few applications.

First step is to augment the `claims.wikibase-item.tsv.gz` file with `derived.P31x.tsv` file which contains occupations for humans as `instance of (P31)`

- `claims.wikibase-item.tsv.gz`: KGTK claims file non literal edges only
- `derived.P31x.tsv`: file with additional P31x links, adding occupation as `instance of` (computed)

In [19]:
!kgtk cat -i $item \
-i $GRAPH/derived.P31x.tsv \
-o $GRAPH/claims.wikibase-item.augmented.tsv.gz

### Run `kgtk graph-embeddings`

The `kgtk graph-embeddings` command takes as input a KGTK edge file and computes graph embeddings of user specified type, producing vectors of user specified dimensions.

The following parameters are used in this instance:

- `-op ComplEx`: compute ComplEx graph embeddings
- `--dimension 30`: desired dimension of the vectors
- `-ot kgtk`: output format - kgtk
- `--retain_temporary_data True`: retain the byproduct files, which we will use in subsequent steps
- `-T <folder path>`: temporary folder where the temporary files will be stored
- `-i <file>`: input file
- `-o <file>`: output file
- `--log <file>`: log file

In [20]:
kgtk(f""" graph-embeddings
            -op ComplEx \
            --dimension $VECTOR_DIMENSION \
            -ot kgtk \
            --retain_temporary_data True \
            -T $TEMP \
            -i $GRAPH/claims.wikibase-item.augmented.tsv.gz \
            -o {vector_output_path} \
            --log $TEMP/ge.log.txt
    """)

In Processing, Please go to /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/ge.log.txt to check details
Processed Finished.



#### Take a peek at the embeddings file.

In [21]:
kgtk(f"""head -i {vector_output_path}""")

,node1,label,node2
0,Q1812591,graph_embeddings,"0.381712556,0.356578857,-0.445301741,-0.575298..."
1,Q15241301,graph_embeddings,"-0.070477232,-0.265077978,-0.877731204,-0.2834..."
2,Q7423761,graph_embeddings,"0.261873245,0.092657343,-0.651605487,-0.284080..."
3,P2437,graph_embeddings,"-0.030844612,-0.434820741,-0.731748879,0.29335..."
4,Q47498954,graph_embeddings,"0.334987372,0.476967394,-0.500187457,0.0323784..."
5,Q4219393,graph_embeddings,"0.150372148,-0.157746866,-0.625900328,0.396662..."
6,Q980308,graph_embeddings,"0.126199126,0.735554039,-0.507171571,-1.141724..."
7,Q3269090,graph_embeddings,"0.041113224,-0.343194246,-0.765003562,0.479651..."
8,Q4026855,graph_embeddings,"-0.143582106,-0.292932570,-0.586739361,-0.0126..."
9,Q1637368,graph_embeddings,"-0.384324908,-0.334103346,-1.244836926,0.17862..."


### The output is in `kgtk` format. Convert it to `word2vec` format for `gensim` similarity computation


For reference: 
- [gensim](https://radimrehurek.com/gensim/)
- [word2vec](https://en.wikipedia.org/wiki/Word2vec)

In [22]:
def convert_kgtk_to_w2v(input_path, output_path):
    """
    Convert a KGTK file (node1/label/node2) that contains embeddings to the w2v format
    """
    vector_count = 0

    # Read the file once to count the lines as we need to put them at the top of the w2v file
    with open(input_path, "r") as kgtk_file:
        next(kgtk_file)
        for line in kgtk_file:
            vector_count += 1
        kgtk_file.close()

    with open(output_path, "w") as w2v_file:
        w2v_file.write("{} {}\n".format(vector_count, vector_dimension))
        with open(input_path, "r") as kgtk_file:
            next(kgtk_file)
            for line in kgtk_file:
                items = line.split("\t")
                qnode = items[0]
                vector = items[2].replace(",", " ")
                w2v_file.write(qnode + " " + vector)
            kgtk_file.close()
        w2v_file.close()

In [23]:
convert_kgtk_to_w2v(f"{vector_output_path}", f"{vector_output_w2v_path}")

### Load the vectors into `gensim`

To find similar vectors based on cosine similarity

In [24]:
ge_vectors = KeyedVectors.load_word2vec_format(f"{vector_output_w2v_path}", binary=False)

Define a function to compute the `topn` similar vectors, and get the labels and descriptions of the matching Qnodes.

In [25]:
def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    add_label_description=True,
    output_path=None,
    topn=25,
):
    """
    find topn similar Qnodes, add label and decription for the Qnodes
    
    :param vectors: vector space loaded into gensim KeyedVectors model
    :param positive: vector(s) or Qnode(s) to find similar entities for
    :param relation_label: name of the property to be used for the output file
    :param add_label_description: boolean parameter to add label and description for matched entities
    :param output_path: path to store the output file
    :param topn: desirednumber of similar entities
    """
    result = []
    if add_label_description:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()

        os.environ["_temp_file"] = filename

        result = !$kypher -i label -i description -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)

### Link Prediction

The following code reads the vectors for Qnodes as `head` and Properties as `relation`.

The files used in the code are produced by `kgtk graph-embeddings` code as a byproduct, in the folder specified by the `-T` option

In [26]:
relation_names_list = json.load(open(f"{os.environ['TEMP']}/output/dynamic_rel_names.json"))
entity_names_list = json.load(open(f"{os.environ['TEMP']}/output/entity_names_all_0.json"))
prop_count = len(relation_names_list)

# operators
operator_lhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
operator_rhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
comparator = DotComparator()
cos_comparator = CosComparator()
with h5py.File(f"{os.environ['TEMP']}/output/model/model.v100.h5", "r") as hf:
    operator_state_dict_lhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/lhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/lhs/imag"][...]),
    }
    operator_state_dict_rhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/rhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/rhs/imag"][...]),
    }
    
operator_lhs.load_state_dict(operator_state_dict_lhs)
operator_rhs.load_state_dict(operator_state_dict_rhs)

# Load the embeddings
with h5py.File(f"{os.environ['TEMP']}/output/model/embeddings_all_0.v100.h5", "r") as hf:
    arnold_embedding = torch.from_numpy(hf["embeddings"][...])


entity_to_index = {}
for i, entity in enumerate(entity_names_list):
    entity_to_index[entity] = i
    

rel_index = {}
for i, rel in enumerate(relation_names_list):
    rel_index[rel] = i

The following function takes as input a `Qnode` and a `Property`, and outputs a vector which should be similar to the value of the relation.

For example, Qnode: `Q37079` = Tom Cruise, Property: `P166` = awards received and output a vector similar to awards. We will see this equation in action in the subsequent examples.

In [27]:
def get_embed(head, relation=None):
    ''' This function generate the embeddings for the tail entities:
            Head entities: Obtained from the model
            Head + relation: Obtained using torch
        :param head: subject Qnode
        :param relation: optional property
    '''
    if relation is None:
        return arnold_embedding[entity_to_index[head], :].detach().numpy()
    return  operator_lhs(
                arnold_embedding[entity_to_index[head], :].view(1, vector_dimension),
                torch.tensor([rel_index[relation]])
            ).detach().numpy()[0]

#### Get the vector for `Q37079` (Tom Cruise) + `P166` (award received), then find most similar entities

In [28]:
_vector = get_embed('Q37079', 'P166')
kgtk_most_similar(ge_vectors, positive=_vector, topn=10)

,node1,node2,label,node1;label,node1;description
0,Q708135,0.16318246722221375,similarity,'Silver Bear'@en,'film award'@en
1,Q1534906,0.12760838866233826,similarity,'Golden Globe Award for New Star of the Year –...,'Golden Globe award'@en
2,Q37079,0.11928117275238037,similarity,'Tom Cruise'@en,'American actor and producer'@en
3,Q268200,0.11478786915540695,similarity,'Screen Actors Guild Award'@en,'accolade given by the Screen Actors Guild‐Ame...
4,Q518675,0.11465701460838318,similarity,'Screen Actors Guild Award for Outstanding Per...,'award'@en
5,Q1011547,0.1022547110915184,similarity,'Golden Globe Award'@en,'award of the Hollywood Foreign Press Associat...
6,Q1413741,0.0845576822757721,similarity,'Saturn Award for Best Actress'@en,'award'@en


#### Get the vector for `Q170564` (Terminator 2: Judgement Day) + `P161` (cast member), then find most similar entities

In [29]:
_vector = get_embed('Q170564', 'P161')
kgtk_most_similar(ge_vectors, positive=_vector, topn=10)

,node1,node2,label,node1;label,node1;description
0,Q330801,0.1517266035079956,similarity,'David Andrews'@en,'actor'@en
1,Q44368,0.14907866716384888,similarity,'André the Giant'@en,'French professional wrestler and actor (1946-...
2,Q236475,0.14035986363887787,similarity,'Olivia Munn'@en,"'American actress, comedian, model, television..."
3,Q16345,0.1395658701658249,similarity,'Leonard Nimoy'@en,"'American actor, film director, poet, musician..."
4,Q47100,0.1281736195087433,similarity,'Ben Stiller'@en,"'American actor, comedian, film producer, dire..."
5,Q781151,0.12697283923625946,similarity,'Austin O\'Brien'@en,'American actor'@en
6,Q355209,0.12432452291250229,similarity,'Richard Schiff'@en,'American actor and director'@en
7,Q676094,0.11988318711519241,similarity,'Drew Barrymore'@en,"'American actress, director and producer'@en"
8,Q229137,0.1191897839307785,similarity,'Karen Allen'@en,'American actress'@en
9,Q320211,0.11842963099479675,similarity,'Stan Winston'@en,'american television and film special make-up ...


#### Get the vector for `Q104123` (Pulp Fiction) + `P161` (cast member), then find most similar entities

In [30]:
_vector = get_embed('Q104123', 'P161')
kgtk_most_similar(ge_vectors, positive=_vector, topn=10)

,node1,node2,label,node1;label,node1;description
0,Q102711,0.17902329564094543,similarity,'Dennis Hopper'@en,'American actor and filmmaker (1936–2010)'@en
1,Q1349205,0.12908358871936798,similarity,'Jason Miller'@en,'American actor and playwright (1939-2001)'@en
2,Q172678,0.11866932362318039,similarity,'Samuel L. Jackson'@en,'American actor and film producer'@en
3,Q106365,0.11587145179510117,similarity,'Isabelle Huppert'@en,'French actress'@en
4,Q2263,0.11583632975816727,similarity,'Tom Hanks'@en,'American actor and producer'@en
5,Q37079,0.1142643392086029,similarity,'Tom Cruise'@en,'American actor and producer'@en
6,Q338726,0.11170054972171783,similarity,'Michael Higgins'@en,'American actor (1920-2008)'@en
7,Q37459,0.10691258311271667,similarity,'Nicole Kidman'@en,'Australian-American actress and film producer...
8,Q298368,0.1026928573846817,similarity,'Liev Schreiber'@en,'American actor'@en
9,Q103784,0.09672869741916656,similarity,'Donald Sutherland'@en,'Canadian actor'@en


#### Get the vector for `Q2685` (Arnold Schwarzenegger), then find most similar entities

In [31]:
_vector = get_embed('Q2685')
kgtk_most_similar(ge_vectors, positive=_vector, topn=10)

,node1,node2,label,node1;label,node1;description
0,Q2685,1.0,similarity,'Arnold Schwarzenegger'@en,"'Austrian-American actor, businessman, bodybui..."
1,Q381178,0.8450968861579895,similarity,'Bob Barker'@en,'American retired television game show host'@en
2,Q1209753,0.8426997661590576,similarity,'Dick Miller'@en,'American actor (1928-2019)'@en
3,Q178166,0.8138990998268127,similarity,'Vin Diesel'@en,"'American actor, producer, director, and scree..."
4,Q102711,0.8135513067245483,similarity,'Dennis Hopper'@en,'American actor and filmmaker (1936–2010)'@en
5,Q461309,0.8132096529006958,similarity,'Doug Jones'@en,"'American actor, contortionist and mime'@en"
6,Q22686,0.8107452988624573,similarity,'Donald Trump'@en,'45th president of the United States'@en
7,Q294901,0.8081139922142029,similarity,'David Caruso'@en,'American actor'@en
8,Q38196234,0.8055974841117859,similarity,'Meinhard Schwarzenegger'@en,'Arnold Schwarzenegger\'s brother'@en


#### Get the vector for `Q103148` (Lahn River), then find most similar entities

In [32]:
_vector = get_embed('Q103148')
kgtk_most_similar(ge_vectors, positive=_vector, topn=10)

,node1,node2,label,node1;label,node1;description
0,Q103148,1.0,similarity,'Lahn'@en,'right tributary of the Rhine River in Germany...
1,Q153945,0.9619489908218384,similarity,'Lippe'@en,'river in Germany'@en
2,Q326366,0.9587880969047546,similarity,'Leimbach'@en,'river'@en
3,Q701645,0.9572846293449402,similarity,'Düssel'@en,'river in Germany'@en
4,Q26727445,0.9552054405212402,similarity,'Moersbach'@en,'river in Germany'@en
5,Q570612,0.9541265368461609,similarity,'Acher'@en,'river in Germany'@en
6,Q319649,0.9493589401245117,similarity,'Möhlin'@en,'river in Germany'@en
7,Q168696,0.948596715927124,similarity,'Nahe'@en,'tributary of Rhine river'@en
8,Q1670,0.9438366889953613,similarity,'Main'@en,'right tributary of Rhine river in Germany'@en
9,Q1673,0.9395939707756042,similarity,'Neckar'@en,'right tributary of Rhine river in Germany'@en


## Prepare files for Google Projector

In this section, we will prepare `vectors` and `metadata` files for google projector.

We are focusing on the following types:

- `Q11424` (film)
- `Q33999` (actor)
- `Q4022` (river)
- `Q82955` (politician)

First step is to create a file with the following information ,

1. node1 :- Qnode
2. label :- name of the property
3. node2 :- embedding vector for node1
4. node1;label :- label for node1
5. type :- `instance of` for node1
6. type;label :- label for type

In [33]:
%%time
kgtk(f""" query -i $GRAPH/claims.wikibase-item.augmented.tsv.gz 
         -i p279star 
         -i label 
         -i {vector_output_path} 
         -i $GRAPH/derived.P31x.tsv 
         --match 'item: (n1)-[]->(), 
             P31x: (n1)-[]->(c), 
             p279star: (c)-[]->(class), 
             label: (n1)-[]->(n1_label), 
             label: (class)-[]->(class_label), embeddings: (n1)-[l]->(embedding)'
        --where 'class in ["Q11424", "Q33999", "Q4022", "Q82955"]' 
        --return 'distinct n1, 
                  l.label as label,
                  embedding as node2,
                  kgtk_lqstring_text(n1_label) as `node1;label`, 
                  group_concat(distinct class) as type, 
                  group_concat(distinct kgtk_lqstring_text(class_label)) as `type;label`'
        -o $TEMP/arnold.embeddings.google.projector.tsv
""")

CPU times: user 6.92 ms, sys: 9.42 ms, total: 16.3 ms
Wall time: 8.39 s


#### Take a peek at the file

In [34]:
kgtk("""head -i $TEMP/arnold.embeddings.google.projector.tsv""")

,node1,label,node2,node1;label,type,type;label
0,Q1000881,graph_embeddings,"-0.700538814,-0.049984712,-0.655169129,0.37428...",Erlau,Q4022,river
1,Q1001872,graph_embeddings,"0.044458453,0.016980033,-0.630594969,0.4194667...",Buersbach,Q4022,river
2,Q1004531,graph_embeddings,"-0.147331789,0.390972495,-0.653173983,0.103166...",Bullets Over Broadway,Q11424,film
3,Q1009788,graph_embeddings,"-0.380878776,0.398105413,-0.479714602,-0.71769...",The Conversation,Q11424,film
4,Q1010099,graph_embeddings,"-0.131890446,0.296854168,-0.636908233,0.246353...",Get Carter,Q11424,film
5,Q1012216,graph_embeddings,"-0.344080091,0.409363985,-0.418321520,-1.32887...",Gorillas in the Mist,Q11424,film
6,Q101410,graph_embeddings,"-0.259945571,0.445760816,-0.568589866,0.083207...",François Fillon,Q82955,politician
7,Q101797,graph_embeddings,"0.076434426,0.729377031,-0.406432807,-1.006446...",Winona Ryder,Q33999,actor
8,Q1018487,graph_embeddings,"-0.315400362,0.392299563,-0.469240367,-0.74686...",Bye Bye Birdie,Q11424,film
9,Q102124,graph_embeddings,"0.299241811,0.630782843,-0.389348388,-0.756123...",Sigourney Weaver,Q33999,actor


#### Define a function to build the required files for google projector

In [35]:
def build_embedding_projector_metadata(gp_embeddings_path, metadata_path, vectors_path):
    """
    build the vector and metadata files required for google projector
    
    :param gp_embeddings_path: file path which has the embeddings and metadata in kgtk format
    :param metadata_path: output file path for metadata
    :param vectors_path: output file path for vectors
    """
    metadata_file = open(metadata_path, "w")
    metadata_file.write("tag\tqnode\ttype\ttype_label\n")

    vectors_file = open(vectors_path, "w")

    with open(gp_embeddings_path) as qnodes_file:
        next(qnodes_file)
        for line in qnodes_file:
            vals = line.split('\t')
            qnode = vals[0]
            qnode_label = vals[3]
            _type = vals[4] 
            ftype_label = vals[5]
            embeddings = "\t".join(vals[2].strip().split(","))

            if qnode.startswith("Q"):
                metadata_file.write("{}\t{}\t{}\t{}\n".format(qnode_label, qnode, _type, ftype_label.strip()))
                vectors_file.write(embeddings)
                vectors_file.write('\n')

    metadata_file.close()
    vectors_file.close()

In [36]:
build_embedding_projector_metadata(f"{os.environ['TEMP']}/arnold.embeddings.google.projector.tsv",
                                  f"{os.environ['OUT']}/arnold.metadata.{vector_dimension}.tsv",
                                  f"{os.environ['OUT']}/arnold.vectors.{vector_dimension}.tsv")

#### Peek at the metadata file

In [37]:
kgtk(f"""head -i $OUT/arnold.metadata.{vector_dimension}.tsv""")

,tag,qnode,type,type_label
0,Erlau,Q1000881,Q4022,river
1,Buersbach,Q1001872,Q4022,river
2,Bullets Over Broadway,Q1004531,Q11424,film
3,The Conversation,Q1009788,Q11424,film
4,Get Carter,Q1010099,Q11424,film
5,Gorillas in the Mist,Q1012216,Q11424,film
6,François Fillon,Q101410,Q82955,politician
7,Winona Ryder,Q101797,Q33999,actor
8,Bye Bye Birdie,Q1018487,Q11424,film
9,Sigourney Weaver,Q102124,Q33999,actor


#### Peek at the vectors file

In [38]:
!head -2 $OUT/arnold.vectors.$VECTOR_DIMENSION.tsv

-0.700538814	-0.049984712	-0.655169129	0.374283344	0.395976931	0.387944371	-0.090598270	0.321922511	-0.178492397	-0.142547786	0.744262636	0.201940536	-0.019090515	-0.168889582	-0.132514343	-0.436341435	-0.480356932	1.389325738	0.102545388	0.453241318	0.218806580	0.109096117	-0.092189699	0.113779262	-0.060277861	0.284524500	0.355568796	-0.118090115	0.016665284	-0.752211571
0.044458453	0.016980033	-0.630594969	0.419466704	-0.186489299	0.476786911	-0.331326991	0.397548884	0.015615340	-0.120036170	0.519031763	-0.235704675	-0.288244307	-0.259112775	-0.298628986	-0.478998065	-0.369969457	1.324729204	-0.148937464	0.936190903	0.198057652	0.239123583	-0.092119031	0.157499775	-0.050901473	0.195211262	0.124998040	-0.446398795	0.139006600	-0.484029740


## Google embedding projector
- open https://projector.tensorflow.org
- Load the vect files using the load button
- configure the visualization

Here we searched on the right for arnold, and we see the closest vecotrs as well as the cluster where it belongs:
![Google embedding projector](assets/gp-arnold.png "Google embedding projector")

#### PCA visualization of the embeddings, colored by `instance of`

![UMAP Color by Type](assets/gp-color-map-types.png "UMAP Color by Type")